In [7]:
from telegram import Update
from telegram.ext import Updater,MessageHandler,Filters, CommandHandler, CallbackContext
import configparser
import logging
import redis

In [8]:
global redis1

In [9]:
def main():
    #Load your token and create an Updater for your Bot
    config=configparser.ConfigParser()
    config.read('config.ini')
    updater=Updater(token=(config['TELEGRAM']['ACCESS_TOKEN']),use_context= True)
    dispatcher=updater.dispatcher
    
    global redis1
    redis1 = redis.Redis(host=(config['REDIS']['HOST']), password=(config['REDIS']['PASSWORD']), port=(config['REDIS']['REDISPORT']))
    
    #You can set this logging module,so you will know when and why things do not work as expected
    logging.basicConfig(format='%(asctime)s-%(name)s-%(levelname)s-%(message)s',level=logging.INFO)
    #register a dispatcher to handle message: here we register an echo dispatcher   
    echo_handler=MessageHandler(Filters.text&(~Filters.command),echo)
    dispatcher.add_handler(echo_handler)
    
    dispatcher.add_handler(CommandHandler("add", add))
    dispatcher.add_handler(CommandHandler("help", help_command))
    
    # TO start the bot
    updater.start_polling()
    updater.idle()

In [10]:
def echo(update, context):
    reply_message = update.message.text.upper()
    logging.info("Update: "+str(update))
    logging.info("Context: "+str(context))
    context.bot.send_message(chat_id = update.effective_chat.id, text = reply_message)

In [11]:
def add(update: Update, context: CallbackContext) -> None:
    try:
        global redis1
        logging.info(context.arg[0])
        msg = context.arg[0]
        # incr 自增 1
        redis1.incr(msg)
        update.message.reply_text('You have said ' + msg + 'for ' + redis1.get(msg).decode('UTF-8') + ' times.')
    except (IndexError, ValueError):
        update.message.reply_text('Usage:/ add <keyword>')

In [12]:
def help_command(update: Update, context: CallbackContext) -> None:
    update.message.reply_text('Helping you helping you')